In [2]:
import os
import logging
working_directory = os.getcwd() + "/"
project_name = "app_vukwm_bag_delivery/"
if project_name in working_directory:
    base_directory = working_directory[:working_directory.find(project_name) + len(project_name)]
    os.chdir(base_directory)
    working_directory = os.getcwd() + "/"
    logging.info(f"Working directory changed to {working_directory}")
else:
    logging.warning(f"Project directory {project_name} not found in {working_directory}")

In [109]:
from GPSOdyssey import Kepler
from keplergl import KeplerGl
import app_vukwm_bag_delivery.aggregates as aggregates
import app_vukwm_bag_delivery.osrm_get_routes as osrm_get_routes
import pandas as pd
import numpy as np

In [110]:
jobs = pd.read_csv("data/test/route_test.csv")
unassigned_stops =  pd.read_csv("data/test/unassigned_jobs.csv")
assigned_stops = pd.read_csv("data/test/route_test.csv")
unassigned_routes = pd.read_csv("data/test/fleet_test.csv")
assigned_stops["Registration No"] = assigned_stops["Vehicle id"]
assigned_stops = assigned_stops[list(unassigned_stops.columns) + ["route_sequence", "Vehicle id"]]
assigned_stops

,Ticket No,Customer Bk,Site Bk,Site Name,Transport Area Code,Product Name,Quantity,Created Date,Required Date,Notes,...,Site Address,collection_date,tansport_area_num,lat,lon,Transport area,Product description,Total boxes,route_sequence,Vehicle id
0,NaN,NaN,NaN,Soho,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,51.513586,-0.137482,NaN,NaN,NaN,0,W04
1,WCSR223461052; WCSR223461056,44921.0,108219.0,Copper Consultancy,2R,Standard bags; Recycling bags,2; 2,2022-12-19,2022-12-20,NaN,...,"COPPER CONSULTANCY, 3RD FLOOR 49 CARNABY STRE...",2022-12-20,2.0,51.512800,-0.138500,# 02,Standard bags: 2\nRecycling bags: 2,4.0,1,W04
2,WCBS223489205,203636.0,498194.0,Rixo LTD,2R,Cardboard Tape,1,2022-12-19,2022-12-20,NaN,...,"RIXO LTD 44 CARNABY STREET LONDON , W1F 9PS",2022-12-20,2.0,51.512800,-0.138700,# 02,Cardboard Tape: 1,1.0,2,W04
3,WCSR223459263,147965.0,331027.0,Z Hotels ZHT Ltd,2R,Standard bags,10,2022-12-19,2022-12-20,NaN,...,"52 POLAND STREET LONDON , W1F 7NQ",2022-12-20,2.0,51.514200,-0.137000,# 02,Standard bags: 10,10.0,3,W04
4,WCBS223537974,204679.0,502342.0,The Milton Group,2R,Standard bags,2,2022-12-19,2022-12-20,NaN,...,"NOLAND HOUSE 12-13 POLAND STREET LONDON , W1...",2022-12-20,2.0,51.514642,-0.136867,# 02,Standard bags: 2,2.0,4,W04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,WCSR223490643,30382.0,278071.0,CAMBRIDGE (BIFFA WASTE SERVICES),1S,Standard bags,8,2022-12-19,2022-12-20,NaN,...,"93 CHARING CROSS ROAD LONDON , WC2H 0DP",2022-12-20,1.0,51.513600,-0.129600,# 01,Standard bags: 8,8.0,22,W05
120,WCSR223514230; WCSR223514263,204472.0,501721.0,Donutelier by Roladin,1S,Standard bags; Recycling bags,6; 4,2022-12-19,2022-12-20,NaN,...,"48-50 CHARING CROSS ROAD LONDON , WC2H 0BB",2022-12-20,1.0,51.512200,-0.128300,# 01,Standard bags: 6\nRecycling bags: 4,10.0,23,W05
121,WCSR223492026,34006.0,106146.0,ROUND TABLE [SWR],1C,Standard bags,4,2022-12-19,2022-12-20,NaN,...,26-27 ST. MARTIN'S COURT CITY OF WESTMINSTE...,2022-12-20,1.0,51.510885,-0.127610,# 01,Standard bags: 4,4.0,24,W05
122,WCBS223261609; WCBS223261608,197025.0,474202.0,Empresa London Limited,1T,Recycling bags; Standard bags,2; 1,2022-12-15,2022-12-16,NaN,...,"EMPRESA LONDON LTD., 25 HENRIETTA STREET LO...",2022-12-16,1.0,51.510800,-0.124300,# 01,Recycling bags: 2\nStandard bags: 1,3.0,25,W05


In [113]:
from shapely import wkt
import app_vukwm_bag_delivery.osrm_get_routes as osrm_get_routes
import pandas as pd
def generate_route_data(assigned_stops, unassigned_routes):

    assigned_stops_route = assigned_stops.merge(
        unassigned_routes.drop(columns=["lat", "lon"])
    )


    assigned_stops_route = assigned_stops_route.assign(
        route_id=assigned_stops_route["Vehicle id"],
        latitude=assigned_stops_route["lat"],
        longitude=assigned_stops_route["lon"],
        vehicle_type=assigned_stops_route["Type"],
        stop_id=np.arange(0, assigned_stops_route.shape[0]),
    )

    unassigned_routes = unassigned_routes.assign(
    route_id=unassigned_routes["Vehicle id"],
    vehicle_type=unassigned_routes["Type"]
)
    osrm_routes = osrm_get_routes.return_route_osrm_info(unassigned_routes, assigned_stops_route)

    travel_legs = osrm_routes[0].copy()

    travel_legs["geometry"] = travel_legs["geometry"].apply(wkt.dumps)
    travel_legs = pd.DataFrame(travel_legs)

    travel_legs["route_sequence"] = travel_legs["travel_sequence"] + 1
    assigned_stops_route_paths = assigned_stops_route.merge(travel_legs, how="left")
    assigned_stops_route_paths["duration_seconds"] = assigned_stops_route_paths[
        "duration_seconds"
    ].fillna(0)
    assigned_stops_route_paths["distance_km"] = assigned_stops_route_paths[
        "distance_km"
    ].fillna(0)

    assigned_stops_route_paths["service_duration_seconds"] = (
        assigned_stops_route_paths["Average TAT per delivery (min)"]
        * 60
        * (~assigned_stops_route_paths["Ticket No"].isna())
    )

    assigned_stops_route_paths["travel_to_service_duration"] = (
        assigned_stops_route_paths["duration_seconds"]
        + assigned_stops_route_paths["service_duration_seconds"]
    )
    assigned_stops_route_paths["total_duration"] = assigned_stops_route_paths.groupby(
        ["route_id"]
    )["travel_to_service_duration"].cumsum()

    assigned_stops_route_paths["depart_stop_seconds"] = assigned_stops_route_paths[
        "total_duration"
    ]
    assigned_stops_route_paths["arrive_stop_seconds"] = (
        assigned_stops_route_paths["total_duration"]
        - assigned_stops_route_paths["service_duration_seconds"]
    )
    assigned_stops_route_paths["arrival_time"] = pd.to_datetime(
        assigned_stops_route_paths["Shift start time"]
    ) + pd.to_timedelta(assigned_stops_route_paths["arrive_stop_seconds"], unit="s")
    assigned_stops_route_paths["depart_time"] = pd.to_datetime(
        assigned_stops_route_paths["Shift start time"]
    ) + pd.to_timedelta(assigned_stops_route_paths["depart_stop_seconds"], unit="s")

    return assigned_stops_route_paths


In [114]:
assigned_stops_route_paths = generate_route_data(assigned_stops, unassigned_routes)

/Users/ejwillemse/dev/waste_labs_dev/app_vukwm_bag_delivery/.venv/lib/python3.10/site-packages/geopandas/geodataframe.py:1442: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,Ticket No,Customer Bk,Site Bk,Site Name,Transport Area Code,Product Name,Quantity,Created Date,Required Date,Notes,...,distance_km,geometry,travel_sequence,service_duration_seconds,travel_to_service_duration,total_duration,depart_stop_seconds,arrive_stop_seconds,arrival_time,depart_time
0,NaN,NaN,NaN,Soho,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0000,NaN,NaN,0,0.0,0.0,0.0,0.0,2022-12-21 09:00:00.000,2022-12-21 09:00:00.000
1,WCSR223461052; WCSR223461056,44921.0,108219.0,Copper Consultancy,2R,Standard bags; Recycling bags,2; 2,2022-12-19,2022-12-20,NaN,...,0.1721,LINESTRING (-0.1373330000000000 51.51362499999...,0.0,300,391.6,391.6,391.6,91.6,2022-12-21 09:01:31.600,2022-12-21 09:06:31.600
2,WCBS223489205,203636.0,498194.0,Rixo LTD,2R,Cardboard Tape,1,2022-12-19,2022-12-20,NaN,...,0.0078,LINESTRING (-0.1385040000000000 51.51279799999...,1.0,300,307.0,698.6,698.6,398.6,2022-12-21 09:06:38.600,2022-12-21 09:11:38.600
3,WCSR223459263,147965.0,331027.0,Z Hotels ZHT Ltd,2R,Standard bags,10,2022-12-19,2022-12-20,NaN,...,0.3126,LINESTRING (-0.1385680000000000 51.51285599999...,2.0,300,450.5,1149.1,1149.1,849.1,2022-12-21 09:14:09.100,2022-12-21 09:19:09.100
4,WCBS223537974,204679.0,502342.0,The Milton Group,2R,Standard bags,2,2022-12-19,2022-12-20,NaN,...,0.0486,LINESTRING (-0.1370380000000000 51.51426699999...,3.0,300,319.6,1468.7,1468.7,1168.7,2022-12-21 09:19:28.700,2022-12-21 09:24:28.700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,WCSR223490643,30382.0,278071.0,CAMBRIDGE (BIFFA WASTE SERVICES),1S,Standard bags,8,2022-12-19,2022-12-20,NaN,...,0.2188,LINESTRING (-0.1304860000000000 51.51350399999...,21.0,300,359.7,9766.8,9766.8,9466.8,2022-12-21 11:37:46.800,2022-12-21 11:42:46.800
120,WCSR223514230; WCSR223514263,204472.0,501721.0,Donutelier by Roladin,1S,Standard bags; Recycling bags,6; 4,2022-12-19,2022-12-20,NaN,...,0.4364,LINESTRING (-0.1293580000000000 51.51365899999...,22.0,300,386.3,10153.1,10153.1,9853.1,2022-12-21 11:44:13.100,2022-12-21 11:49:13.100
121,WCSR223492026,34006.0,106146.0,ROUND TABLE [SWR],1C,Standard bags,4,2022-12-19,2022-12-20,NaN,...,0.2250,LINESTRING (-0.1284700000000000 51.51217299999...,23.0,300,358.0,10511.1,10511.1,10211.1,2022-12-21 11:50:11.100,2022-12-21 11:55:11.100
122,WCBS223261609; WCBS223261608,197025.0,474202.0,Empresa London Limited,1T,Recycling bags; Standard bags,2; 1,2022-12-15,2022-12-16,NaN,...,0.4472,LINESTRING (-0.1270120000000000 51.51096400000...,24.0,300,400.0,10911.1,10911.1,10611.1,2022-12-21 11:56:51.100,2022-12-21 12:01:51.100


In [87]:
travel_legs = osrm_routes[0].copy()
travel_legs.shape[0]

119

In [88]:
travel_legs["geometry"] = travel_legs["geometry"].apply(wkt.dumps)
travel_legs = pd.DataFrame(travel_legs)

/Users/ejwillemse/dev/waste_labs_dev/app_vukwm_bag_delivery/.venv/lib/python3.10/site-packages/geopandas/geodataframe.py:1442: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [63]:
assigned_stops_route.columns

Index(['Ticket No', 'Customer Bk', 'Site Bk', 'Site Name',
       'Transport Area Code', 'Product Name', 'Quantity', 'Created Date',
       'Required Date', 'Notes', 'on hold', 'Schedule ID', 'Scheduled Date',
       'Completed Date', 'Registration No', 'Site Address1', 'Site Address2',
       'Site Address3', 'Site Address4', 'Site Address5', 'Site Post Town',
       'Site Post Code', 'Site Latitude', 'Site Longitude', 'Site Address',
       'collection_date', 'tansport_area_num', 'lat', 'lon', 'Transport area',
       'Product description', 'Total boxes', 'route_sequence', 'Vehicle id',
       'Type', 'Capacity (#boxes)', 'Depot', 'Shift start time',
       'Average TAT per delivery (min)', 'Dedicated transport zones',
       'Cost (£) per km', 'Cost (£) per hour', 'route_id', 'latitude',
       'longitude', 'vehicle_type', 'stop_id'],
      dtype='object')

In [89]:
travel_legs["route_sequence"] = travel_legs["travel_sequence"] + 1
assigned_stops_route_paths = assigned_stops_route.merge(travel_legs, how="left")
assigned_stops_route_paths["duration_seconds"] = assigned_stops_route_paths[
    "duration_seconds"
].fillna(0)
assigned_stops_route_paths["distance_km"] = assigned_stops_route_paths[
    "distance_km"
].fillna(0)

assigned_stops_route_paths["service_duration_seconds"] = (
    assigned_stops_route_paths["Average TAT per delivery (min)"]
    * 60
    * (~assigned_stops_route_paths["Ticket No"].isna())
)

assigned_stops_route_paths["travel_to_service_duration"] = (
    assigned_stops_route_paths["duration_seconds"]
    + assigned_stops_route_paths["service_duration_seconds"]
)
assigned_stops_route_paths["total_duration"] = assigned_stops_route_paths.groupby(
    ["route_id"]
)["travel_to_service_duration"].cumsum()

assigned_stops_route_paths["depart_stop_seconds"] = assigned_stops_route_paths[
    "total_duration"
]
assigned_stops_route_paths["arrive_stop_seconds"] = (
    assigned_stops_route_paths["total_duration"]
    - assigned_stops_route_paths["service_duration_seconds"]
)
assigned_stops_route_paths["arrival_time"] = pd.to_datetime(
    assigned_stops_route_paths["Shift start time"]
) + pd.to_timedelta(assigned_stops_route_paths["arrive_stop_seconds"], unit="s")
assigned_stops_route_paths["depart_time"] = pd.to_datetime(
    assigned_stops_route_paths["Shift start time"]
) + pd.to_timedelta(assigned_stops_route_paths["depart_stop_seconds"], unit="s")


In [105]:
def return_order_config():
    config = {
        "version": "v1",
        "config": {
            "visState": {
                "filters": [
                    {
                        "dataId": ["orders"],
                        "id": "yt10dkouq",
                        "name": ["arrival_time"],
                        "type": "timeRange",
                        "value": [1671613200000, 1671626666000],
                        "enlarged": True,
                        "plotType": "histogram",
                        "animationWindow": "free",
                        "yAxis": None,
                        "speed": 1,
                    },
                    {
                        "dataId": ["orders"],
                        "id": "wrkhad0sp",
                        "name": ["Registration No"],
                        "type": "multiSelect",
                        "value": [],
                        "enlarged": False,
                        "plotType": "histogram",
                        "animationWindow": "free",
                        "yAxis": None,
                        "speed": 1,
                    },
                ],
                "layers": [
                    {
                        "id": "o4a1e2",
                        "type": "point",
                        "config": {
                            "dataId": "orders",
                            "label": "Stops",
                            "color": [241, 92, 23],
                            "highlightColor": [252, 242, 26, 255],
                            "columns": {
                                "lat": "Site Latitude",
                                "lng": "Site Longitude",
                                "altitude": None,
                            },
                            "isVisible": True,
                            "visConfig": {
                                "radius": 15,
                                "fixedRadius": False,
                                "opacity": 0.81,
                                "outline": True,
                                "thickness": 2,
                                "strokeColor": [25, 20, 16],
                                "colorRange": {
                                    "name": "Uber Viz Qualitative 4",
                                    "type": "qualitative",
                                    "category": "Uber",
                                    "colors": [
                                        "#12939A",
                                        "#DDB27C",
                                        "#88572C",
                                        "#FF991F",
                                        "#F15C17",
                                        "#223F9A",
                                        "#DA70BF",
                                        "#125C77",
                                        "#4DC19C",
                                        "#776E57",
                                        "#17B8BE",
                                        "#F6D18A",
                                        "#B7885E",
                                        "#FFCB99",
                                        "#F89570",
                                        "#829AE3",
                                        "#E79FD5",
                                        "#1E96BE",
                                        "#89DAC1",
                                        "#B3AD9E",
                                    ],
                                },
                                "strokeColorRange": {
                                    "name": "Global Warming",
                                    "type": "sequential",
                                    "category": "Uber",
                                    "colors": [
                                        "#5A1846",
                                        "#900C3F",
                                        "#C70039",
                                        "#E3611C",
                                        "#F1920E",
                                        "#FFC300",
                                    ],
                                },
                                "radiusRange": [0, 50],
                                "filled": True,
                            },
                            "hidden": False,
                            "textLabel": [
                                {
                                    "field": {
                                        "name": "route_sequence",
                                        "type": "integer",
                                    },
                                    "color": [205, 201, 191],
                                    "size": 18,
                                    "offset": [0, 0],
                                    "anchor": "start",
                                    "alignment": "center",
                                }
                            ],
                        },
                        "visualChannels": {
                            "colorField": {"name": "Registration No", "type": "string"},
                            "colorScale": "ordinal",
                            "strokeColorField": None,
                            "strokeColorScale": "quantile",
                            "sizeField": None,
                            "sizeScale": "linear",
                        },
                    },
                    {
                        "id": "h5ysd8s",
                        "type": "geojson",
                        "config": {
                            "dataId": "orders",
                            "label": "Routes",
                            "color": [255, 203, 153],
                            "highlightColor": [252, 242, 26, 255],
                            "columns": {"geojson": "geometry"},
                            "isVisible": True,
                            "visConfig": {
                                "opacity": 0.8,
                                "strokeOpacity": 0.8,
                                "thickness": 0.5,
                                "strokeColor": None,
                                "colorRange": {
                                    "name": "Global Warming",
                                    "type": "sequential",
                                    "category": "Uber",
                                    "colors": [
                                        "#5A1846",
                                        "#900C3F",
                                        "#C70039",
                                        "#E3611C",
                                        "#F1920E",
                                        "#FFC300",
                                    ],
                                },
                                "strokeColorRange": {
                                    "name": "Uber Viz Qualitative 4",
                                    "type": "qualitative",
                                    "category": "Uber",
                                    "colors": [
                                        "#12939A",
                                        "#DDB27C",
                                        "#88572C",
                                        "#FF991F",
                                        "#F15C17",
                                        "#223F9A",
                                        "#DA70BF",
                                        "#125C77",
                                        "#4DC19C",
                                        "#776E57",
                                        "#17B8BE",
                                        "#F6D18A",
                                        "#B7885E",
                                        "#FFCB99",
                                        "#F89570",
                                        "#829AE3",
                                        "#E79FD5",
                                        "#1E96BE",
                                        "#89DAC1",
                                        "#B3AD9E",
                                    ],
                                },
                                "radius": 10,
                                "sizeRange": [0, 10],
                                "radiusRange": [0, 50],
                                "heightRange": [0, 500],
                                "elevationScale": 5,
                                "enableElevationZoomFactor": True,
                                "stroked": True,
                                "filled": False,
                                "enable3d": False,
                                "wireframe": False,
                            },
                            "hidden": False,
                            "textLabel": [
                                {
                                    "field": None,
                                    "color": [255, 255, 255],
                                    "size": 18,
                                    "offset": [0, 0],
                                    "anchor": "start",
                                    "alignment": "center",
                                }
                            ],
                        },
                        "visualChannels": {
                            "colorField": None,
                            "colorScale": "quantile",
                            "strokeColorField": {
                                "name": "Registration No",
                                "type": "string",
                            },
                            "strokeColorScale": "ordinal",
                            "sizeField": None,
                            "sizeScale": "linear",
                            "heightField": None,
                            "heightScale": "linear",
                            "radiusField": None,
                            "radiusScale": "linear",
                        },
                    },
                ],
                "interactionConfig": {
                    "tooltip": {
                        "fieldsToShow": {
                            "orders": [
                                {"name": "Customer Bk", "format": None},
                                {"name": "Site Bk", "format": None},
                                {"name": "Site Name", "format": None},
                                {"name": "Transport Area Code", "format": None},
                                {"name": "Site Address", "format": None},
                                {"name": "Notes", "format": None},
                                {"name": "Product description", "format": None},
                                {"name": "Registration No", "format": None},
                                {"name": "route_sequence", "format": None},
                                {"name": "Transport area", "format": None},
                                {"name": "Type", "format": None},
                                {"name": "arrival_time", "format": None},
                                {"name": "depart_time", "format": None},
                            ]
                        },
                        "compareMode": False,
                        "compareType": "absolute",
                        "enabled": True,
                    },
                    "brush": {"size": 0.5, "enabled": False},
                    "geocoder": {"enabled": False},
                    "coordinate": {"enabled": False},
                },
                "layerBlending": "normal",
                "splitMaps": [],
                "animationConfig": {"currentTime": None, "speed": 1},
            },
            "mapState": {
                "bearing": 0,
                "dragRotate": False,
                "latitude": 51.503679000596435,
                "longitude": -0.11381729292045753,
                "pitch": 0,
                "zoom": 12.5,
                "isSplit": False,
            },
            "mapStyle": {
                "styleType": "dark",
                "topLayerGroups": {},
                "visibleLayerGroups": {
                    "label": True,
                    "road": True,
                    "border": False,
                    "building": True,
                    "water": True,
                    "land": True,
                    "3d building": False,
                },
                "threeDBuildingColor": [
                    9.665468314072013,
                    17.18305478057247,
                    31.1442867897876,
                ],
                "mapStyles": {},
            },
        },
    }
    return config


In [106]:
routes_plot = assigned_stops_route_paths.copy()

In [107]:
m = Kepler(data={"orders": routes_plot}, config=return_order_config(), height=750)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/Users/ejwillemse/dev/waste_labs_dev/app_vukwm_bag_delivery/.venv/lib/python3.10/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [108]:
m.get_render()

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['orders'], 'id': 'yt10dkouq'…

In [3]:
from app_vukwm_bag_delivery.return_routes import generate_route_data
import pandas as pd

In [4]:
unassigned_stops =  pd.read_csv("data/test/unassigned_jobs.csv")
assigned_stops = pd.read_csv("data/test/route_test.csv")
unassigned_routes = pd.read_csv("data/test/fleet_test.csv")

route_data = generate_route_data(unassigned_stops, unassigned_routes, unassigned_stops)

KeyError: 'Vehicle id'